In [1]:
import sys
import findspark
findspark.init()
import pyspark
from pyspark import SparkContext, SparkConf
import os

In [2]:

conf = SparkConf().setAppName("Read Log files").setMaster("local[*]")
sc = SparkContext(conf = conf)


In [3]:
# A regular expression pattern to extract fields from the log line
#APACHE_ACCESS_LOG_PATTERN = '^(\S+) (\S+) (\S+) \[([\w:/]+\s[+\-]\d{4})\] "(\S+) (\S+)\s*(\S*)" (\d{3}) (\S+)'

APACHE_ACCESS_LOG_PATTERN = '^(\S+) (\S+) (\S+) \[([\w:/]+\s[+\-]\d{4})\] "(\S+) (.*?) (\S+)" (\d{3}) (\S+) "(.*?)" "(.*?)"$'


In [4]:
import re
import datetime

from pyspark.sql import Row

month_map = {'Jan': 1, 'Feb': 2, 'Mar':3, 'Apr':4, 'May':5, 'Jun':6, 'Jul':7,
    'Aug':8,  'Sep': 9, 'Oct':10, 'Nov': 11, 'Dec': 12}

def parse_apache_time(s):
    """ Convert Apache time format into a Python datetime object
    Args:
        s (str): date and time in Apache time format
    Returns:
        datetime: datetime object (ignore timezone for now)
    """
    return datetime.datetime(int(s[7:11]),
                             month_map[s[3:6]],
                             int(s[0:2]),
                             int(s[12:14]),
                             int(s[15:17]),
                             int(s[18:20]))


def parseApacheLogLine(logline):
    """ Parse a line in the Apache Common Log format
    Args:
        logline (str): a line of text in the Apache Common Log format
    Returns:
        tuple: either a dictionary containing the parts of the Apache Access Log and 1,
               or the original invalid log line and 0
    """
    match = re.search(APACHE_ACCESS_LOG_PATTERN, logline)
    if match is None:
        return (logline, 0)
    size_field = match.group(9)
    if size_field == '-':
        size = 0
    else:
        size = match.group(9)
    return (Row(
        host          = match.group(1),
        client_identd = match.group(2),
        user_id       = match.group(3),
        date_time     = parse_apache_time(match.group(4)),
        method        = match.group(5),
        endpoint      = match.group(6),
        protocol      = match.group(7),
        response_code = int(match.group(8)),
        content_size  = size,
        referrer     = match.group(10),
        userAgent    = match.group(11)
    ), 1)



In [5]:
logFile = "apache_logs.txt"

def parseLogs():
    """ Read and parse log file """
    parsed_logs = (sc.textFile(logFile).map(parseApacheLogLine).cache())

    access_logs = (parsed_logs
                   .filter(lambda s: s[1] == 1)
                   .map(lambda s: s[0])
                   .cache())

    failed_logs = (parsed_logs
                   .filter(lambda s: s[1] == 0)
                   .map(lambda s: s[0]))
    failed_logs_count = failed_logs.count()
    if failed_logs_count > 0:
        print ('Number of invalid logline: %d' % failed_logs.count())
        for line in failed_logs.take(20):
            print ('Invalid logline: %s' % line)

    print ('Read %d lines, successfully parsed %d lines, failed to parse %d lines' % (parsed_logs.count(), access_logs.count(), failed_logs.count()))
    return parsed_logs, access_logs, failed_logs


parsed_logs, access_logs, failed_logs = parseLogs()




Number of invalid logline: 2
Invalid logline: 83.149.9.216 - - [17/May/2015:10:05:03 +0000] "GET /presentations/logstash-monitorama-2013/images/kibana-search.png HTTP/1.1" 200 203023 "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/32.0.1700.77 Safari/537.36"
Invalid logline: 46.118.127.106 - - [20/May/2015:12:05:17 +0000] "GET /scripts/grok-py-test/configlib.py HTTP/1.1" 200 235 "-" "Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html
Read 10000 lines, successfully parsed 9998 lines, failed to parse 2 lines


In [6]:
print(access_logs.take(2))

[Row(host='83.149.9.216', client_identd='-', user_id='-', date_time=datetime.datetime(2015, 5, 17, 10, 5, 43), method='GET', endpoint='/presentations/logstash-monitorama-2013/images/kibana-dashboard3.png', protocol='HTTP/1.1', response_code=200, content_size='171717', referrer='http://semicomplete.com/presentations/logstash-monitorama-2013/', userAgent='Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/32.0.1700.77 Safari/537.36'), Row(host='83.149.9.216', client_identd='-', user_id='-', date_time=datetime.datetime(2015, 5, 17, 10, 5, 47), method='GET', endpoint='/presentations/logstash-monitorama-2013/plugin/highlight/highlight.js', protocol='HTTP/1.1', response_code=200, content_size='26185', referrer='http://semicomplete.com/presentations/logstash-monitorama-2013/', userAgent='Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/32.0.1700.77 Safari/537.36')]


In [29]:
access_logs.groupBy(lambda word: word.host)
print(access_logs.take(2))

[Row(host='83.149.9.216', client_identd='-', user_id='-', date_time=datetime.datetime(2015, 5, 17, 10, 5, 43), method='GET', endpoint='/presentations/logstash-monitorama-2013/images/kibana-dashboard3.png', protocol='HTTP/1.1', response_code=200, content_size='171717', referrer='http://semicomplete.com/presentations/logstash-monitorama-2013/', userAgent='Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/32.0.1700.77 Safari/537.36'), Row(host='83.149.9.216', client_identd='-', user_id='-', date_time=datetime.datetime(2015, 5, 17, 10, 5, 47), method='GET', endpoint='/presentations/logstash-monitorama-2013/plugin/highlight/highlight.js', protocol='HTTP/1.1', response_code=200, content_size='26185', referrer='http://semicomplete.com/presentations/logstash-monitorama-2013/', userAgent='Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/32.0.1700.77 Safari/537.36')]


In [43]:
#total session time

times_date = access_logs.map(lambda s: s.date_time)
bSorted = times_date.sortBy(lambda a: a)

first_time = bSorted.first()
totalCount = bSorted.count()
last_time = bSorted.collect()[totalCount - 1 ]
total_session_time = last_time - first_time

print("Total session time: ", total_session_time)

Total session time:  3 days, 11:00:59


In [15]:
#total session time

g_rdd = access_logs.map(lambda s: (s.host, s.date_time))
r_rdd = g_rdd.groupBy(lambda w: w[0])

#print(r_rdd.take(10))
print(r_rdd.collect()[0][1])
for t in r_rdd.collect()[0][1]:
    print(t)

('83.149.9.216', datetime.datetime(2015, 5, 17, 10, 5, 43))
('83.149.9.216', datetime.datetime(2015, 5, 17, 10, 5, 47))
('83.149.9.216', datetime.datetime(2015, 5, 17, 10, 5, 12))
('83.149.9.216', datetime.datetime(2015, 5, 17, 10, 5, 7))
('83.149.9.216', datetime.datetime(2015, 5, 17, 10, 5, 34))
('83.149.9.216', datetime.datetime(2015, 5, 17, 10, 5, 57))
('83.149.9.216', datetime.datetime(2015, 5, 17, 10, 5, 50))
('83.149.9.216', datetime.datetime(2015, 5, 17, 10, 5, 24))
('83.149.9.216', datetime.datetime(2015, 5, 17, 10, 5, 50))
('83.149.9.216', datetime.datetime(2015, 5, 17, 10, 5, 46))
('83.149.9.216', datetime.datetime(2015, 5, 17, 10, 5, 11))
('83.149.9.216', datetime.datetime(2015, 5, 17, 10, 5, 19))
('83.149.9.216', datetime.datetime(2015, 5, 17, 10, 5, 33))
('83.149.9.216', datetime.datetime(2015, 5, 17, 10, 5))
('83.149.9.216', datetime.datetime(2015, 5, 17, 10, 5, 25))
('83.149.9.216', datetime.datetime(2015, 5, 17, 10, 5, 59))
('83.149.9.216', datetime.datetime(2015, 5, 1

In [19]:
for t in access_logs.collect():
        print(t[0], t[1],t[2],t[3])

83.149.9.216 - - 2015-05-17 10:05:43
83.149.9.216 - - 2015-05-17 10:05:47
83.149.9.216 - - 2015-05-17 10:05:12
83.149.9.216 - - 2015-05-17 10:05:07
83.149.9.216 - - 2015-05-17 10:05:34
83.149.9.216 - - 2015-05-17 10:05:57
83.149.9.216 - - 2015-05-17 10:05:50
83.149.9.216 - - 2015-05-17 10:05:24
83.149.9.216 - - 2015-05-17 10:05:50
83.149.9.216 - - 2015-05-17 10:05:46
83.149.9.216 - - 2015-05-17 10:05:11
83.149.9.216 - - 2015-05-17 10:05:19
83.149.9.216 - - 2015-05-17 10:05:33
83.149.9.216 - - 2015-05-17 10:05:00
83.149.9.216 - - 2015-05-17 10:05:25
83.149.9.216 - - 2015-05-17 10:05:59
83.149.9.216 - - 2015-05-17 10:05:30
83.149.9.216 - - 2015-05-17 10:05:53
83.149.9.216 - - 2015-05-17 10:05:24
83.149.9.216 - - 2015-05-17 10:05:54
83.149.9.216 - - 2015-05-17 10:05:33
83.149.9.216 - - 2015-05-17 10:05:56
24.236.252.67 - - 2015-05-17 10:05:40
93.114.45.13 - - 2015-05-17 10:05:14
93.114.45.13 - - 2015-05-17 10:05:04
93.114.45.13 - - 2015-05-17 10:05:45
93.114.45.13 - - 2015-05-17 10:05:14


65.55.213.74 - - 2015-05-17 14:05:53
65.55.213.74 - - 2015-05-17 14:05:09
65.55.213.74 - - 2015-05-17 14:05:05
81.169.149.220 - - 2015-05-17 14:05:13
65.55.213.74 - - 2015-05-17 14:05:25
65.55.213.79 - - 2015-05-17 14:05:21
65.55.213.79 - - 2015-05-17 14:05:05
65.55.213.79 - - 2015-05-17 14:05:55
65.55.213.79 - - 2015-05-17 14:05:04
65.55.213.79 - - 2015-05-17 14:05:10
65.55.213.79 - - 2015-05-17 14:05:42
65.55.213.73 - - 2015-05-17 14:05:25
65.55.213.73 - - 2015-05-17 14:05:58
65.55.213.73 - - 2015-05-17 14:05:06
65.55.213.73 - - 2015-05-17 14:05:03
65.55.213.73 - - 2015-05-17 14:05:25
65.55.213.73 - - 2015-05-17 14:05:31
65.55.213.73 - - 2015-05-17 14:05:26
65.55.213.73 - - 2015-05-17 14:05:34
65.55.213.73 - - 2015-05-17 14:05:33
65.55.213.73 - - 2015-05-17 14:05:11
65.55.213.73 - - 2015-05-17 14:05:50
65.55.213.73 - - 2015-05-17 14:05:44
65.55.213.73 - - 2015-05-17 14:05:28
65.55.213.73 - - 2015-05-17 14:05:39
65.55.213.73 - - 2015-05-17 14:05:38
65.55.213.73 - - 2015-05-17 14:05:05

116.125.143.84 - - 2015-05-17 16:05:22
116.125.143.84 - - 2015-05-17 16:05:35
100.43.83.137 - - 2015-05-17 16:05:53
54.219.224.86 - - 2015-05-17 16:05:59
108.171.116.194 - - 2015-05-17 16:05:17
99.188.185.40 - - 2015-05-17 16:05:10
108.171.116.194 - - 2015-05-17 16:05:04
108.171.116.194 - - 2015-05-17 16:05:37
108.171.116.194 - - 2015-05-17 16:05:02
108.171.116.194 - - 2015-05-17 16:05:47
108.171.116.194 - - 2015-05-17 16:05:18
108.171.116.194 - - 2015-05-17 16:05:08
108.171.116.194 - - 2015-05-17 16:05:53
108.171.116.194 - - 2015-05-17 16:05:55
108.171.116.194 - - 2015-05-17 16:05:31
202.65.130.3 - - 2015-05-17 16:05:23
46.105.14.53 - - 2015-05-17 16:05:07
68.180.224.225 - - 2015-05-17 16:05:59
201.244.101.132 - - 2015-05-17 16:05:27
193.238.231.119 - - 2015-05-17 16:05:28
197.206.232.180 - - 2015-05-17 16:05:34
180.76.6.53 - - 2015-05-17 16:05:29
207.241.237.226 - - 2015-05-17 16:05:49
109.87.146.211 - - 2015-05-17 16:05:19
166.82.99.140 - - 2015-05-17 16:05:24
216.14.102.16 - - 2015

66.249.73.135 - - 2015-05-17 19:05:56
62.161.94.37 - - 2015-05-17 19:05:19
66.249.73.135 - - 2015-05-17 19:05:25
46.105.14.53 - - 2015-05-17 19:05:48
67.220.144.164 - - 2015-05-17 19:05:34
67.220.144.164 - - 2015-05-17 19:05:22
98.248.53.169 - - 2015-05-17 19:05:26
98.248.53.169 - - 2015-05-17 19:05:24
98.248.53.169 - - 2015-05-17 19:05:21
98.248.53.169 - - 2015-05-17 19:05:30
98.248.53.169 - - 2015-05-17 19:05:38
93.152.153.38 - - 2015-05-17 19:05:17
93.152.153.38 - - 2015-05-17 19:05:49
93.152.153.38 - - 2015-05-17 19:05:40
93.152.153.38 - - 2015-05-17 19:05:31
93.152.153.38 - - 2015-05-17 19:05:56
146.1.1.2 - - 2015-05-17 19:05:41
146.1.1.2 - - 2015-05-17 19:05:31
54.196.174.134 - - 2015-05-17 19:05:09
50.16.19.13 - - 2015-05-17 19:05:04
208.93.0.48 - - 2015-05-17 19:05:40
208.91.156.11 - - 2015-05-17 19:05:00
146.1.1.2 - - 2015-05-17 19:05:45
146.1.1.2 - - 2015-05-17 19:05:08
146.1.1.2 - - 2015-05-17 19:05:21
146.1.1.2 - - 2015-05-17 19:05:39
88.196.179.78 - - 2015-05-17 19:05:17
6

49.204.238.249 - - 2015-05-17 21:05:21
49.204.238.249 - - 2015-05-17 21:05:38
80.237.234.150 - - 2015-05-17 21:05:31
46.105.14.53 - - 2015-05-17 21:05:10
209.85.238.199 - - 2015-05-17 21:05:30
74.125.176.148 - - 2015-05-17 21:05:18
182.118.20.213 - - 2015-05-17 21:05:31
100.43.83.137 - - 2015-05-17 21:05:57
66.249.73.185 - - 2015-05-17 21:05:01
208.115.111.72 - - 2015-05-17 21:05:06
99.252.100.83 - - 2015-05-17 21:05:10
99.252.100.83 - - 2015-05-17 21:05:48
99.252.100.83 - - 2015-05-17 21:05:31
99.252.100.83 - - 2015-05-17 21:05:05
99.252.100.83 - - 2015-05-17 21:05:38
99.252.100.83 - - 2015-05-17 21:05:28
99.252.100.83 - - 2015-05-17 21:05:00
99.252.100.83 - - 2015-05-17 21:05:31
99.252.100.83 - - 2015-05-17 21:05:16
99.252.100.83 - - 2015-05-17 21:05:04
99.252.100.83 - - 2015-05-17 21:05:28
99.252.100.83 - - 2015-05-17 21:05:25
99.252.100.83 - - 2015-05-17 21:05:39
99.252.100.83 - - 2015-05-17 21:05:25
99.252.100.83 - - 2015-05-17 21:05:20
99.252.100.83 - - 2015-05-17 21:05:06
99.252

...

109.231.204.82 - - 2015-05-20 10:05:27
209.85.238.199 - - 2015-05-20 10:05:35
46.105.14.53 - - 2015-05-20 10:05:30
80.130.134.87 - - 2015-05-20 10:05:10
80.130.134.87 - - 2015-05-20 10:05:23
200.10.161.5 - - 2015-05-20 10:05:22
200.10.161.5 - - 2015-05-20 10:05:59
92.236.181.94 - - 2015-05-20 10:05:58
92.236.181.94 - - 2015-05-20 10:05:05
92.236.181.94 - - 2015-05-20 10:05:47
92.236.181.94 - - 2015-05-20 10:05:44
92.236.181.94 - - 2015-05-20 10:05:56
92.236.181.94 - - 2015-05-20 10:05:16
65.19.138.33 - - 2015-05-20 10:05:17
66.249.73.135 - - 2015-05-20 10:05:42
50.16.19.13 - - 2015-05-20 10:05:58
209.17.114.78 - - 2015-05-20 11:05:08
24.0.194.37 - - 2015-05-20 11:05:00
180.76.5.189 - - 2015-05-20 11:05:03
46.105.14.53 - - 2015-05-20 11:05:50
24.0.194.37 - - 2015-05-20 11:05:15
24.0.194.37 - - 2015-05-20 11:05:19
24.0.194.37 - - 2015-05-20 11:05:00
24.0.194.37 - - 2015-05-20 11:05:16
24.0.194.37 - - 2015-05-20 11:05:01
24.0.194.37 - - 2015-05-20 11:05:12
24.0.194.37 - - 2015-05-20 11:05

189.11.65.66 - - 2015-05-20 13:05:16
189.11.65.66 - - 2015-05-20 13:05:19
189.11.65.66 - - 2015-05-20 13:05:42
189.11.65.66 - - 2015-05-20 13:05:19
80.160.68.134 - - 2015-05-20 13:05:27
66.249.73.135 - - 2015-05-20 13:05:04
188.192.27.241 - - 2015-05-20 13:05:02
208.115.111.72 - - 2015-05-20 13:05:18
157.55.33.15 - - 2015-05-20 13:05:32
184.151.222.45 - - 2015-05-20 13:05:57
46.105.14.53 - - 2015-05-20 13:05:28
66.249.73.135 - - 2015-05-20 13:05:19
173.213.99.1 - - 2015-05-20 13:05:47
173.213.99.1 - - 2015-05-20 13:05:42
180.76.5.22 - - 2015-05-20 13:05:58
208.91.156.11 - - 2015-05-20 13:05:55
50.16.19.13 - - 2015-05-20 13:05:51
198.27.64.9 - - 2015-05-20 13:05:48
187.45.193.158 - - 2015-05-20 13:05:36
68.180.224.225 - - 2015-05-20 13:05:34
198.27.64.9 - - 2015-05-20 13:05:16
128.118.108.67 - - 2015-05-20 13:05:01
128.179.155.97 - - 2015-05-20 13:05:41
66.249.73.135 - - 2015-05-20 13:05:18
68.14.231.140 - - 2015-05-20 13:05:50
128.179.155.97 - - 2015-05-20 13:05:07
216.172.140.128 - - 

107.170.9.55 - - 2015-05-20 15:05:05
74.125.40.20 - - 2015-05-20 15:05:48
49.206.120.190 - - 2015-05-20 15:05:57
49.206.120.190 - - 2015-05-20 15:05:32
49.206.120.190 - - 2015-05-20 15:05:21
49.206.120.190 - - 2015-05-20 15:05:10
49.206.120.190 - - 2015-05-20 15:05:37
82.60.18.23 - - 2015-05-20 15:05:31
82.60.18.23 - - 2015-05-20 15:05:32
5.10.83.65 - - 2015-05-20 15:05:27
46.105.14.53 - - 2015-05-20 15:05:43
24.190.3.5 - - 2015-05-20 15:05:57
74.76.53.142 - - 2015-05-20 15:05:21
66.249.73.135 - - 2015-05-20 15:05:10
68.180.224.225 - - 2015-05-20 15:05:16
68.180.224.225 - - 2015-05-20 15:05:49
173.162.242.85 - - 2015-05-20 15:05:02
68.180.224.225 - - 2015-05-20 15:05:32
128.118.108.67 - - 2015-05-20 15:05:15
119.224.20.139 - - 2015-05-20 15:05:38
74.76.53.142 - - 2015-05-20 15:05:57
74.76.53.142 - - 2015-05-20 15:05:01
74.76.53.142 - - 2015-05-20 15:05:55
74.76.53.142 - - 2015-05-20 15:05:15
74.76.53.142 - - 2015-05-20 15:05:53
66.249.73.135 - - 2015-05-20 15:05:47
68.180.224.225 - - 2

141.142.223.45 - - 2015-05-20 18:05:02
5.10.83.23 - - 2015-05-20 18:05:00
182.253.73.95 - - 2015-05-20 18:05:37
182.253.73.95 - - 2015-05-20 18:05:06
78.47.134.185 - - 2015-05-20 18:05:41
66.249.73.135 - - 2015-05-20 18:05:12
46.105.14.53 - - 2015-05-20 18:05:09
50.16.19.13 - - 2015-05-20 18:05:38
5.10.83.82 - - 2015-05-20 18:05:15
92.100.97.83 - - 2015-05-20 18:05:02
92.100.97.83 - - 2015-05-20 18:05:54
23.229.67.14 - - 2015-05-20 18:05:05
99.146.78.102 - - 2015-05-20 18:05:40
23.229.67.14 - - 2015-05-20 18:05:13
190.107.140.178 - - 2015-05-20 18:05:49
5.10.83.23 - - 2015-05-20 18:05:00
66.249.73.135 - - 2015-05-20 18:05:46
208.43.243.244 - - 2015-05-20 18:05:40
208.43.243.244 - - 2015-05-20 18:05:36
208.43.243.244 - - 2015-05-20 18:05:28
208.43.243.244 - - 2015-05-20 18:05:34
15.219.153.83 - - 2015-05-20 18:05:11
15.219.153.83 - - 2015-05-20 18:05:29
15.219.153.83 - - 2015-05-20 18:05:13
180.76.5.73 - - 2015-05-20 18:05:07
208.43.243.244 - - 2015-05-20 18:05:59
216.151.137.35 - - 201

120.136.4.243 - - 2015-05-20 21:05:13
66.249.73.185 - - 2015-05-20 21:05:45
195.194.187.106 - - 2015-05-20 21:05:22
66.249.73.185 - - 2015-05-20 21:05:09
66.249.73.185 - - 2015-05-20 21:05:32
176.31.39.30 - - 2015-05-20 21:05:42
176.31.39.30 - - 2015-05-20 21:05:17
116.199.211.249 - - 2015-05-20 21:05:16
46.105.14.53 - - 2015-05-20 21:05:39
5.10.83.53 - - 2015-05-20 21:05:07
66.249.73.135 - - 2015-05-20 21:05:59
92.115.179.247 - - 2015-05-20 21:05:46
92.115.179.247 - - 2015-05-20 21:05:31
92.115.179.247 - - 2015-05-20 21:05:35
92.115.179.247 - - 2015-05-20 21:05:01
92.115.179.247 - - 2015-05-20 21:05:34
92.115.179.247 - - 2015-05-20 21:05:35
5.10.83.53 - - 2015-05-20 21:05:59
46.119.114.245 - - 2015-05-20 21:05:28
173.231.106.34 - - 2015-05-20 21:05:19
66.169.220.99 - - 2015-05-20 21:05:03
66.249.73.135 - - 2015-05-20 21:05:37
50.16.19.13 - - 2015-05-20 21:05:43
5.10.83.21 - - 2015-05-20 21:05:36
208.91.156.11 - - 2015-05-20 21:05:05
66.249.73.135 - - 2015-05-20 21:05:18
66.249.73.135 

In [32]:
parsed_logs2.groupBy(lambda word: word[0])
parsed_logs2.take(2)

['83.149.9.216 - - [17/May/2015:10:05:03 +0000] "GET /presentations/logstash-monitorama-2013/images/kibana-search.png HTTP/1.1" 200 203023 "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/32.0.1700.77 Safari/537.36"',
 '83.149.9.216 - - [17/May/2015:10:05:43 +0000] "GET /presentations/logstash-monitorama-2013/images/kibana-dashboard3.png HTTP/1.1" 200 171717 "http://semicomplete.com/presentations/logstash-monitorama-2013/" "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/32.0.1700.77 Safari/537.36"']

In [33]:
parsed_logs2 = (sc.textFile(logFile))
parsed_logs2.take(2)

['83.149.9.216 - - [17/May/2015:10:05:03 +0000] "GET /presentations/logstash-monitorama-2013/images/kibana-search.png HTTP/1.1" 200 203023 "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/32.0.1700.77 Safari/537.36"',
 '83.149.9.216 - - [17/May/2015:10:05:43 +0000] "GET /presentations/logstash-monitorama-2013/images/kibana-dashboard3.png HTTP/1.1" 200 171717 "http://semicomplete.com/presentations/logstash-monitorama-2013/" "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/32.0.1700.77 Safari/537.36"']

In [8]:
parsed_logs2.take(1)[0]

'83.149.9.216 - - [17/May/2015:10:05:03 +0000] "GET /presentations/logstash-monitorama-2013/images/kibana-search.png HTTP/1.1" 200 203023 "http://semicomplete.com/presentations/logstash-monitorama-2013/" "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/32.0.1700.77 Safari/537.36"'

In [22]:
not200 = access_logs.filter(lambda log: log.responseCode != 200)


AttributeError: 'PipelinedRDD' object has no attribute 'show'

In [21]:
parsedGroupBy.take(1)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 15.0 failed 1 times, most recent failure: Lost task 0.0 in stage 15.0 (TID 25, host.docker.internal, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "D:\spark\spark-3.0.1-bin-hadoop2.7\python\lib\pyspark.zip\pyspark\worker.py", line 605, in main
  File "D:\spark\spark-3.0.1-bin-hadoop2.7\python\lib\pyspark.zip\pyspark\worker.py", line 595, in process
  File "D:\spark\spark-3.0.1-bin-hadoop2.7\python\pyspark\rdd.py", line 2596, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "D:\spark\spark-3.0.1-bin-hadoop2.7\python\pyspark\rdd.py", line 2596, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "D:\spark\spark-3.0.1-bin-hadoop2.7\python\pyspark\rdd.py", line 425, in func
    return f(iterator)
  File "D:

Total session time:  3 days, 11:00:59


In [11]:
logFile = "apache_logs.txt"
file = sc.textFile(logFile)

In [13]:
print(file.first())

83.149.9.216 - - [17/May/2015:10:05:03 +0000] "GET /presentations/logstash-monitorama-2013/images/kibana-search.png HTTP/1.1" 200 203023 "http://semicomplete.com/presentations/logstash-monitorama-2013/" "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/32.0.1700.77 Safari/537.36"


In [3]:
# A regular expression pattern to extract fields from the log line
APACHE_ACCESS_LOG_PATTERN = '^(\S+) (\S+) (\S+) \[([\w:/]+\s[+\-]\d{4})\] "(\S+) (\S+)\s*(\S*)" (\d{3}) (\S+)'

In [4]:
import re
import datetime

from pyspark.sql import Row

month_map = {'Jan': 1, 'Feb': 2, 'Mar':3, 'Apr':4, 'May':5, 'Jun':6, 'Jul':7,
    'Aug':8,  'Sep': 9, 'Oct':10, 'Nov': 11, 'Dec': 12}

def parse_apache_time(s):
    """ Convert Apache time format into a Python datetime object
    Args:
        s (str): date and time in Apache time format
    Returns:
        datetime: datetime object (ignore timezone for now)
    """
    return datetime.datetime(int(s[7:11]),
                             month_map[s[3:6]],
                             int(s[0:2]),
                             int(s[12:14]),
                             int(s[15:17]),
                             int(s[18:20]))


def parseApacheLogLine(logline):
    """ Parse a line in the Apache Common Log format
    Args:
        logline (str): a line of text in the Apache Common Log format
    Returns:
        tuple: either a dictionary containing the parts of the Apache Access Log and 1,
               or the original invalid log line and 0
    """
    match = re.search(APACHE_ACCESS_LOG_PATTERN, logline)
    if match is None:
        return (logline, 0)
    size_field = match.group(9)
    if size_field == '-':
        size = 0
    else:
        size = match.group(9)
    return (Row(
        host          = match.group(1),
        client_identd = match.group(2),
        user_id       = match.group(3),
        date_time     = parse_apache_time(match.group(4)),
        method        = match.group(5),
        endpoint      = match.group(6),
        protocol      = match.group(7),
        response_code = int(match.group(8)),
        content_size  = size
    ), 1)



In [11]:
logFile = "apache_logs.txt"
parsed_logs = (sc.textFile(logFile).map(parseApacheLogLine).cache())
access_logs = (parsed_logs.filter(lambda s: s[1] == 1).map(lambda s: s[0]).cache())
failed_logs = (parsed_logs.filter(lambda s: s[1] == 0).map(lambda s: s[0]))
failed_logs_count = failed_logs.count()

if failed_logs_count > 0:
    print ('Number of invalid logline: %d' % failed_logs.count())
    for line in failed_logs.take(20):
        print ('Invalid logline: %s' % line)

print ('Read %d lines, successfully parsed %d lines, failed to parse %d lines' % (parsed_logs.count(), access_logs.count(), failed_logs.count()))


Read 10000 lines, successfully parsed 10000 lines, failed to parse 0 lines


In [10]:
print(failed_logs.count())

0


In [18]:
logFile = "apache_logs.txt"

def parseLogs():
    """ Read and parse log file """
    parsed_logs = (sc.textFile(logFile).map(parseApacheLogLine).cache())

    access_logs = (parsed_logs
                   .filter(lambda s: s[1] == 1)
                   .map(lambda s: s[0])
                   .cache())

    failed_logs = (parsed_logs
                   .filter(lambda s: s[1] == 0)
                   .map(lambda s: s[0]))
    failed_logs_count = failed_logs.count()
    if failed_logs_count > 0:
        print ('Number of invalid logline: %d' % failed_logs.count())
        for line in failed_logs.take(20):
            print ('Invalid logline: %s' % line)

    print ('Read %d lines, successfully parsed %d lines, failed to parse %d lines' % (parsed_logs.count(), access_logs.count(), failed_logs.count()))
    return parsed_logs, access_logs, failed_logs


parsed_logs, access_logs, failed_logs = parseLogs()




Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 2.0 failed 1 times, most recent failure: Lost task 1.0 in stage 2.0 (TID 4, host.docker.internal, executor driver): java.net.SocketException: Connection reset
	at java.net.SocketInputStream.read(SocketInputStream.java:209)
	at java.net.SocketInputStream.read(SocketInputStream.java:141)
	at java.io.BufferedInputStream.fill(BufferedInputStream.java:246)
	at java.io.BufferedInputStream.read(BufferedInputStream.java:265)
	at java.io.DataInputStream.readInt(DataInputStream.java:387)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:628)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:621)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:456)
	at org.apache.spark.InterruptibleIterator.hasNext(Interrup

In [8]:
  
import re
from pyspark.sql import Row

In [9]:
APACHE_ACCESS_LOG_PATTERN = '^(\S+) (\S+) (\S+) \[([\w:/]+\s[+\-]\d{4})\] "(\S+) (\S+) (\S+)" (\d{3}) (\d+)'

In [ ]:
# Returns a dictionary containing the parts of the Apache Access Log.
def parse_apache_log_line(logline):
    match = re.search(APACHE_ACCESS_LOG_PATTERN, logline)
    if match is None:
        raise Error("Invalid logline: %s" % logline)
    return Row(
        ip_address    = match.group(1),
        client_identd = match.group(2),
        user_id       = match.group(3),
        date_time     = match.group(4),
        method        = match.group(5),
        endpoint      = match.group(6),
        protocol      = match.group(7),
        response_code = int(match.group(8)),
        content_size  = long(match.group(9))
    )